### **Import Required Libraries**

In [51]:
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder
from sklearn.metrics import accuracy_score,classification_report,precision_score,recall_score,f1_score
from sklearn.model_selection import GridSearchCV

### **Import Datasets**

In [52]:
train_data=pd.read_csv("../dataset/train_data.csv")
test_data=pd.read_csv("../dataset/test_data.csv")

In [53]:
print(f"Shape of the Training data:{train_data.shape}")
print(f"Shape of the Testing data:{test_data.shape}")

Shape of the Training data:(1550, 13)
Shape of the Testing data:(388, 13)


### **Drop Unecessary Columns**

In [54]:
train_data=train_data.drop(["Unnamed: 0"],axis=1)
test_data=test_data.drop(["Unnamed: 0"],axis=1)

In [55]:
##split train data into the X_train and Y_train
X_train=train_data.drop(["delivery_status"],axis=1)
Y_train=train_data["delivery_status"]

In [56]:
##Adjust the datatype of  assembly_service_requested column
X_train["assembly_service_requested"]=X_train["assembly_service_requested"].astype("object")

### **Model Building**

In [57]:
# ### Numbercal columns 
numerical_features=X_train.select_dtypes(include="number").columns.to_list()
categorical_features=X_train.select_dtypes(include="object").columns.to_list()

# ##Nominal variables
categorical_features.remove("brand")
##Ordinal variables
ordinal_features=["brand"]

In [58]:
### Define pipelines
nominal_pipeline=Pipeline(steps=[
    ("One-Hot-Encoder",OneHotEncoder(sparse_output=False,handle_unknown="ignore"))
])
ordinal_pipeline=Pipeline(steps=[
    ("Ordinal-Encoder",OrdinalEncoder())
])

transfomer=ColumnTransformer(transformers=[
    ("Nominal Pipeline",nominal_pipeline,categorical_features),
    ("Ordinal Pipeline",ordinal_pipeline,ordinal_features)
]
)

final_pipeline=Pipeline(steps=[
    ("Transfomer",transfomer),
    ("ABC", AdaBoostClassifier(DecisionTreeClassifier(random_state=42)))
])

## Execute Pipeline
final_pipeline.fit(X_train,Y_train)

,steps,"[('Transfomer', ...), ('ABC', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('Nominal Pipeline', ...), ('Ordinal Pipeline', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


### **Model Evaluation**

In [59]:
##split test data into the X_test and Y_test
X_test=test_data.drop(["delivery_status"],axis=1)
Y_test=test_data["delivery_status"]

##Adjust the datatype of  assembly_service_requested column
X_test["assembly_service_requested"]=X_test["assembly_service_requested"].astype("object")

In [60]:
y_pred=final_pipeline.predict(X_test) ##get the predicted values

In [61]:
##print the classification report
print(classification_report(Y_test,y_pred))

                 precision    recall  f1-score   support

      Delivered       0.21      0.12      0.15        69
Failed Delivery       0.48      0.66      0.56       192
       On Going       0.31      0.21      0.25       127

       accuracy                           0.41       388
      macro avg       0.33      0.33      0.32       388
   weighted avg       0.38      0.41      0.38       388



In [62]:
y_pred_train=final_pipeline.predict(X_train)

In [63]:
##print the classification report
print(classification_report(Y_train,y_pred_train))

                 precision    recall  f1-score   support

      Delivered       0.55      0.28      0.37       275
Failed Delivery       0.60      0.83      0.70       767
       On Going       0.59      0.40      0.48       508

       accuracy                           0.59      1550
      macro avg       0.58      0.51      0.52      1550
   weighted avg       0.59      0.59      0.57      1550



In [64]:
### precision,recall,f1-score for test data and train data
precision_test=precision_score(Y_test,y_pred,average="weighted")
recall_test=recall_score(Y_test,y_pred,average="weighted")
f1_test=f1_score(Y_test,y_pred,average="weighted")

precision_train=precision_score(Y_train,y_pred_train,average="weighted")
recall_train=recall_score(Y_train,y_pred_train,average="weighted")
f1_train=f1_score(Y_train,y_pred_train,average="weighted")


In [65]:
##print the precision,recall,f1-score
print(f"Test Data - Precision: {precision_test}, Recall: {recall_test}, F1-Score: {f1_test}")
print(f"Train Data - Precision: {precision_train}, Recall: {recall_train}, F1-Score: {f1_train}")


Test Data - Precision: 0.3757974495287862, Recall: 0.41494845360824745, F1-Score: 0.38383464861003713
Train Data - Precision: 0.5889452759719549, Recall: 0.5941935483870968, F1-Score: 0.5685715242035776


### **Hyper Parameter Tune**

In [66]:
##define parameters for GridSearchCV
param_grid = {
    "ABC__n_estimators": [50, 100, 200],
    "ABC__learning_rate": [0.01, 0.1, 1.0],
    "ABC__estimator__max_depth": [1, 2, 3]
}

In [67]:
grid_search = GridSearchCV(
    final_pipeline,
    param_grid=param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
    verbose=2
)
grid_search.fit(X_train, Y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


,estimator,Pipeline(step..._state=42)))])
,param_grid,"{'ABC__estimator__max_depth': [1, 2, ...], 'ABC__learning_rate': [0.01, 0.1, ...], 'ABC__n_estimators': [50, 100, ...]}"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('Nominal Pipeline', ...), ('Ordinal Pipeline', ...)]"


In [68]:
##best parameters
print("Best Parameters:",grid_search.best_params_)

Best Parameters: {'ABC__estimator__max_depth': 1, 'ABC__learning_rate': 0.01, 'ABC__n_estimators': 50}


### **Model Evaluation**

In [69]:
# Evaluate best model
best_rf =grid_search.best_estimator_
y_pred_cv = best_rf.predict(X_test)

In [70]:
print("\nTest Accuracy:", accuracy_score(Y_test, y_pred_cv))
print("\nClassification Report:\n", classification_report(Y_test, y_pred_cv))


Test Accuracy: 0.4948453608247423

Classification Report:
                  precision    recall  f1-score   support

      Delivered       0.00      0.00      0.00        69
Failed Delivery       0.49      1.00      0.66       192
       On Going       0.00      0.00      0.00       127

       accuracy                           0.49       388
      macro avg       0.16      0.33      0.22       388
   weighted avg       0.24      0.49      0.33       388



c:\Users\ASUS\anaconda3\envs\pyfix\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ASUS\anaconda3\envs\pyfix\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ASUS\anaconda3\envs\pyfix\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0]

In [71]:
y_pred_train_cv=best_rf.predict(X_train)

In [72]:
##print the classification report
print("\nTrain Accuracy:", accuracy_score(Y_train, y_pred_train_cv))
print(classification_report(Y_train,y_pred_train_cv))


Train Accuracy: 0.49483870967741933
                 precision    recall  f1-score   support

      Delivered       0.00      0.00      0.00       275
Failed Delivery       0.49      1.00      0.66       767
       On Going       0.00      0.00      0.00       508

       accuracy                           0.49      1550
      macro avg       0.16      0.33      0.22      1550
   weighted avg       0.24      0.49      0.33      1550



c:\Users\ASUS\anaconda3\envs\pyfix\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ASUS\anaconda3\envs\pyfix\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ASUS\anaconda3\envs\pyfix\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0]

In [73]:
##precision,recall,f1-score for test data and train data
precision_test_cv=precision_score(Y_test,y_pred_cv,average="weighted")
recall_test_cv=recall_score(Y_test,y_pred_cv,average="weighted")    
f1_test_cv=f1_score(Y_test,y_pred_cv,average="weighted")

precision_train_cv=precision_score(Y_train,y_pred_train_cv,average="weighted")
recall_train_cv=recall_score(Y_train,y_pred_train_cv,average="weighted")
f1_train_cv=f1_score(Y_train,y_pred_train_cv,average="weighted")


c:\Users\ASUS\anaconda3\envs\pyfix\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ASUS\anaconda3\envs\pyfix\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [74]:
##print the precision,recall,f1-score
print(f"Test Data (CV) - Precision: {precision_test_cv}, Recall: {recall_test_cv}, F1-Score: {f1_test_cv}")
print(f"Train Data (CV) - Precision: {precision_train_cv}, Recall: {recall_train_cv}, F1-Score: {f1_train_cv}")


Test Data (CV) - Precision: 0.24487193112976938, Recall: 0.4948453608247423, F1-Score: 0.32762175613224315
Train Data (CV) - Precision: 0.2448653485952133, Recall: 0.49483870967741933, F1-Score: 0.32761440683865395
